In [348]:
import json
from typing import List, Tuple

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import MeCab
import MySQLdb

In [349]:
# DB utils
def get_connector_and_cursor(host_name: str) \
    -> Tuple[pd.core.frame.DataFrame, MySQLdb.cursors.Cursor]:
    conn = MySQLdb.connect(
        host = host_name,
        port = 3306,
        user = 'root',
        password = 'root',
        database = 'maindb',
        use_unicode=True,
        charset="utf8"
    )
    cursor = conn.cursor()
    return conn, cursor


# Point prediction utils
def count_noun_number(mecab: MeCab.Tagger, text: str) -> int:
    count = []
    for line in mecab.parse(str(text)).splitlines():
        try:
            if "名詞" in line.split()[-1]:
                count.append(line)
        except:
            pass
    return len(set(count))


def preprocessing(detail_df: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    '''ポイント予測のために新しい特徴量を作成する。
    
    Fields:
        title_length: タイトルの文字数
        story_length: あらすじの文字数
        text_length: 本文の文字数
        keyword_number: キーワードの数
        noun_proportion_in_text: 本文中における文字数当たりの名詞数
    '''
    mecab = MeCab.Tagger("-Ochasen")
    
    for column in ['title', 'story', 'text']:
        detail_df[column + '_length'] = detail_df[column].apply(lambda x: len(str(x)))
    detail_df['keyword_number'] = detail_df['keyword'].apply(lambda x: len(str(x).split(' ')))
    detail_df['noun_proportion_in_text'] = detail_df.text.apply(
            lambda x: count_noun_number(mecab, str(x)) / len(str(x))
    )
    return detail_df


def point_prediction(url: str, detail_df: pd.core.frame.DataFrame) -> List[int]:
    detail_df = preprocessing(detail_df)
    
    headers = {'Content-Type': 'application/json'}
    data = {}
    data = {column: list(detail_df[column]) for column in list(detail_df.columns)}
    data = json.dumps(data)
    r_post = requests.post(url, headers=headers, json=data)

    predicted_points = r_post.json()['prediction']
    return predicted_points


# Feature extraction utils
def _generate_data(ncodes: List[str], features: List[float]) -> dict:
    for ncode, feature in zip(ncodes, features):
        yield {
            '_index': 'features',
            'ncode': ncode,
            'feature': feature
        }


def extract_features(url: str, texts: List[str]) -> List[float]:
    headers = {'Content-Type': 'application/json'}
    data = {'texts': texts}
    r_post = requests.post(url, headers=headers, json=data)
    features = r_post.json()['prediction']
    return features


def add_features_to_elasticsearch(client: elasticsearch.client.Elasticsearch, url: str, ncodes: List[str], texts: List[str], h_dim: int=64):
    features = extract_features(url, texts)
    
    if not client.indices.exists(index='features'):
        mappings = {
            'properties': {
                'ncode': {'type': 'keyword'},
                'feature': {'type': 'dense_vector', 'dims': h_dim}
            }
        }
        client.indices.create(index='features', body={'mappings': mappings })
    
    bulk(client, _generate_data(ncodes, features))

In [350]:
TEST = True
MINIBATCH_SIZE = 10
NAROU_API_URL = 'https://api.syosetu.com/novelapi/api/'

# ELASTICSEARCH_HOST_NAME = 'elasticsearch'
# FEATURE_EXTRACTION_URL = 'http://bertserver:3032/predict'
# POINT_PREDICTION_URL = 'http://mlserver:3033/predict'
# DB_HOST_NAME = 'database'

ELASTICSEARCH_HOST_NAME = 'localhost:9200'
FEATURE_EXTRACTION_URL = 'http://localhost:3032/predict'
POINT_PREDICTION_URL = 'http://localhost:3033/predict'
DB_HOST_NAME = '0.0.0.0'

In [351]:
class DetailsScraper(object):
    """作品の詳細情報をスクレイピングするためのクラス
    
    Methods:
        scraping_details: 作品の詳細情報をスクレイピングするgenerator。
                                       mode='first'では最古の作品から最新の作品が対象。
                                       mode='middle'ではDB内で最新の作品から最新の作品が対象。
        __details_preprocessing: 作品の詳細情報を格納したDataFrameに対して前処理を行う。
        __get_scraping_item_number: スクレイピングの対象となる作品数を計算する。
    """
    
    def __init__(self, narou_api_url: str, batch_size: int=64, interval: fsloat= 0.1, mode: str='first'):
        self.narou_api_url = narou_api_url
        self.batch_size = batch_size
        self.interval = interval
        self.mode = mode
        self.now = str(int(datetime.datetime.now().timestamp()))
        
    def scraping_details(self, cursor: MySQLdb.cursors.Cursor=None) -> pd.core.frame.DataFrame:

        if self.mode == 'first':
            latest_registered_datetime = '1073779200'
        elif self.mode == 'middle' and cursor:
            cursor.execute("SELECT general_lastup FROM details ORDER BY general_lastup DESC LIMIT 1")
            sql_result = cursor.fetchone()
            latest_registered_datetime = str(sql_result[0]) if sql_result is not None else '1073779200'
        else:
            raise Exception('Argument mode should be middle or first.')

        lastup = self.now
        allcount = self.__get_scraping_item_number(latest_registered_datetime)
        all_queue_cnt = (allcount // self.batch_size)

        for i in range(all_queue_cnt):
            payload = {
                'out': 'json', 
                'gzip': 5, 
                'opt': 'weekly',
                'lim': self.batch_size,
                'lastup': latest_registered_datetime+"-"+str(lastup)
            }

            c = 0 # Avoid infinite loop
            while c < 5:
                try:
                    res = requests.get(self.narou_api_url, params=payload, timeout=30).content
                    break
                except:
                    print('Connection Error')
                    c += 1       

            r = gzip.decompress(res).decode('utf-8')

            details_df = pd.read_json(r).drop(0)
            details_df = self.__details_preprocessing(details_df)

            lastup = details_df.iloc[-1]["general_lastup"]

            time.sleep(self.interval)
            yield details_df

    def __details_preprocessing(self, details_df: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
        details_df = details_df.drop(['allcount', 'gensaku'], axis=1, errors='ignore')
        details_df = details_df.dropna(how='all')

        date_to_timestamp = lambda date: int(datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S").timestamp())

        for column in details_df.columns:
            if column in ['title', 'ncode', 'userid', 'writer', 'story', 'keyword']:
                details_df[column] = details_df[column].astype(str)
            elif column in['general_firstup', 'general_lastup', 'novelupdated_at', 'updated_at']:
                details_df[column] = details_df[column].map(str).map(date_to_timestamp)
            else:
                details_df[column] = details_df[column].astype(int)

        details_df['predict_point'] = 'Nan'
        details_df['text'] = 'Nan'

        return details_df

    def __get_scraping_item_number(self, latest_registered_datetime: str) -> int:
        payload = {
            'out': 'json', 
            'gzip': 5, 
            'of': 'n',
            'lim': 1, 
            'lastup': latest_registered_datetime+"-"+self.now
        }
        res = requests.get(self.narou_api_url, params=payload).content
        r =  gzip.decompress(res).decode("utf-8") 
        allcount = json.loads(r)[0]['allcount']
        return allcount

In [352]:
class TextScraper(object):
    """作品の本文をスクレイピングするためのクラス
    
    Methods:
        scraping_texts: 作品の本文をスクレイピングする。短編か連載かによって処理が異なる。
        __make_bs_obj: 引数のURLのBeautifulSoupオブジェクトを作成する。
        __get_main_text: BeautifulSoupオブジェクトを元に作品の本文をタグによって抽出する。
    """
    
    def __init__(self, narou_api_url: str, interval :float=0.1):
        self.narou_api_url = narou_api_url
        self.interval = interval
        
    def scraping_texts(self, ncodes: List[str]) -> Tuple[List[str]]:
        texts = []
        processed_ncodes = []

        for ncode in ncodes:
            time.sleep(self.interval)
            url = 'https://ncode.syosetu.com/' + ncode + '/'
            
            c = 0 # Avoid infinite loop
            while c < 5:
                try:
                    bs_obj = self.__make_bs_obj(url)
                    if bs_obj.findAll("dl", {"class": "novel_sublist2"}): # 連載作品の場合
                        url = 'https://ncode.syosetu.com/' + ncode + '/1/'
                        bs_obj = self.__make_bs_obj(url)
                    text = self.__get_main_text(bs_obj)  
                    break
                except Exception as e:
                    print(e)
                    c += 1
        
            processed_ncodes.append(ncode)
            texts.append(text)   
        return processed_ncodes, texts

    def __make_bs_obj(self, url: str) -> str:
        html = urlopen(url)
        return BeautifulSoup(html,"html.parser")

    def __get_main_text(self, bs_obj: BeautifulSoup) -> str:
        text = ""
        text_htmls = bs_obj.findAll('div', {'id': 'novel_honbun'})[0].findAll("p")

        for text_html in text_htmls:
            text = text + text_html.get_text() + "\n"

        return text

In [424]:
class Scraper(object):
    """スクレイピングとデータの投入を行うクラス
    
    Methods:
        scraping_and_add: 作品の詳細と本文をスクレイピングしデータの投入を行う。
        __add_to_database: 作品の詳細情報をDatabaseへ投入する。
        __add_to_elasticsearch: 作品の本文から抽出された特徴量をElasticsearchへ投入する。
    """
    
    def __init__(self, batch_size_of_scraper: int=64, batch_size_of_es: int=16, test=True):
        self.batch_size_of_scraper = batch_size_of_scraper
        self.batch_size_of_es = batch_size_of_es
        self.test = test
        
        self.conn, self.cursor = get_connector_and_cursor(DB_HOST_NAME)
        self.client = Elasticsearch(ELASTICSEARCH_HOST_NAME)
        self.details_scraper = DetailsScraper(NAROU_API_URL, batch_size_of_scraper, mode='first')
        self.text_scraper = TextScraper(NAROU_API_URL)
        
    def scraping_and_add(self):
        scraping_details_iterator = self.details_scraper.scraping_details()
        for i, details_df in enumerate(scraping_details_iterator):
            print((i + 1) * self.batch_size_of_scraper)
            ncodes, texts = self.text_scraper.scraping_texts(details_df.ncode)
            for ncode, text in zip(ncodes, texts):
                details_df.loc[details_df['ncode'] == ncode, 'text'] = text[:1024]
            predict_point = point_prediction(POINT_PREDICTION_URL, details_df)
            details_df['predict_point'] = predict_point
        
            self.__add_to_database(details_df)
            self.__add_to_elasticsearch(details_df)
            
            if self.test:
                break
        
        self.conn.close()
        self.client.close()
        
    def add_existing_data(self):
        if self.test:
            details_df_iterator = pd.read_sql_query("SELECT * FROM details LIMIT 64", conn, chunksize=32)
        else:
            details_df_iterator = pd.read_sql_query("SELECT * FROM details WHERE predict_point='Nan'", conn, chunksize=32)
        for details_df in details_df_iterator:
            predict_point = point_prediction(POINT_PREDICTION_URL, details_df)
            details_df['predict_point'] = predict_point
            
            self.__update_database(details_df.ncode, details_df.predict_point)
            self.__add_to_elasticsearch(details_df)
            
            if self.test:
                break
        
        self.conn.close()
        self.client.close()
        
    def __add_to_database(self, details_df):
        self.cursor.execute("SHOW columns FROM details")
        columns_of_details = [column[0] for column in cursor.fetchall()]
        details_df_tmp = details_df[columns_of_details]
        details_data = [tuple(details_df_tmp.iloc[i]) for i in range(len(details_df_tmp))]
        self.cursor.executemany("INSERT IGNORE INTO details VALUES ({})".format(("%s, "*len(columns_of_details))[:-2]), details_data)
        self.conn.commit()
        
    def __update_database(self, ncodes, predict_points):
        data = [(ncode, predict_point) for ncode, predict_point in zip(ncodes, predict_points)]
        self.cursor.executemany("UPDATE details SET predict_point=%s WHERE ncode=%s", data)
        self.conn.commit()
        
    def __add_to_elasticsearch(self, details_df):
        recommendable_df = details_df[(details_df['predict_point'] == 1) & (details_df['global_point'] == 0)]
        if len(recommendable_df) != 0:
            ncodes, texts = list(recommendable_df.ncode), list(recommendable_df.text)
            
            for i in range(len(ncodes) // self.batch_size_of_es + 1):
                start, end = i * self.batch_size_of_es, (i + 1) * self.batch_size_of_es
                add_features_to_elasticsearch(
                    self.client, 
                    FEATURE_EXTRACTION_URL, 
                    ncodes[start:end], 
                    texts[start:end]
                )
        print('{} data is inserted to Elasticsearch.'.format(len(recommendable_df)))
        
    def __del__(self):
        try:
            self.conn.close()
            self.client.close()
        except:
            pass

In [425]:
scraper = Scraper()

In [426]:
scraper.scraping_and_add()

64


KeyboardInterrupt: 

In [427]:
scraper.add_existing_data()

7 data is inserted to Elasticsearch.


In [430]:
type(conn)

MySQLdb.connections.Connection

In [376]:
cursor.execute("SHOW columns FROM details")

41

In [377]:
columns_of_details = [column[0] for column in cursor.fetchall()]

In [378]:
details_df_tmp = details_df[columns_of_details]

In [379]:
details_data = [tuple(details_df_tmp.iloc[i]) for i in range(len(details_df_tmp))]

In [381]:
cursor.executemany("INSERT IGNORE INTO details VALUES ({})".format(("%s, "*len(columns_of_details))[:-2]), details_data)

64

In [382]:
conn.commit()

In [406]:
df_iterator = pd.read_sql_query("SELECT * FROM details", conn, chunksize=2)

In [407]:
for df in df_iterator:
    print(df)

                                   title    ncode     userid writer  \
0  かれこれ１６年ほど忌み子をやっておりました。１７年目に魔王に攫われました。  N5866FF   727711.0     瓶覗   
1                              荒くれ騎士の嘆き歌  N1272GI  1221860.0    OWL   

                                               story  biggenre  genre  \
0  魔神を身の内に封印した伝説を持つ王家で、魔神を抑えきれない「災厄を呼ぶ者」が生まれた。\n産...         2    201   
1  騎士は嘆く\n所詮自分は殺し屋だと。\n騎士道は殺しの正当化の手段に過ぎないと。\n\n姫は...         2    201   

                                             keyword  general_firstup  \
0          R15 残酷な描写あり 冒険 魔王 魔法 忌み子 人攫い魔王 旅する魔王 食べ歩き       1546482524   
1  R15 異類婚姻譚 身分差 年の差 悲恋 ヒストリカル シリアス ダークファンタジー 男主人...       1592827348   

   general_lastup  ...  review_cnt  all_point  all_hyoka_cnt  sasie_cnt  \
0      1598772598  ...           0         60              7          0   
1      1598772582  ...           0         10              1          0   

   kaiwaritu  novelupdated_at  updated_at  weekly_unique  \
0         45       1598772598  1598772724              0   


[2 rows x 41 columns]
                                   title    ncode    userid writer  \
0                                    あわい  N9645DM  818958.0   藤野 羊   
1  かれこれ１６年ほど忌み子をやっておりました。１７年目に魔王に攫われました。  N5866FF  727711.0     瓶覗   

                                               story  biggenre  genre  \
0  “人生に代えても欲しいものがある”。\nそれは例えば、今いる場所から逃げること。不遇な生涯の...         2    202   
1  魔神を身の内に封印した伝説を持つ王家で、魔神を抑えきれない「災厄を呼ぶ者」が生まれた。\n産...         2    201   

                                             keyword  general_firstup  \
0  R15 残酷な描写あり 日常 異能力バトル 一次創作 現代 長編 ファンタジー ライトノベル...       1473006292   
1          R15 残酷な描写あり 冒険 魔王 魔法 忌み子 人攫い魔王 旅する魔王 食べ歩き       1546482524   

   general_lastup  ...  review_cnt  all_point  all_hyoka_cnt  sasie_cnt  \
0      1598772632  ...           0          9              2          0   
1      1598772598  ...           0         60              7          0   

   kaiwaritu  novelupdated_at  updated_at  weekly_unique  \
0         36       1598772632  1598772844

[2 rows x 41 columns]
                                              title    ncode     userid  \
0  万色を支配する白魔王　～ステータス極振りどころか全捨てし、スキルに全振りした少女のピーキー無双～  N6320GL  1434253.0   
1                    DREADNOTES～それは異端者が世界の理不尽と戦う物語～  N3085FN  1646730.0   

  writer                                              story  biggenre  genre  \
0   志水零士  ＶＲＭＭＯの適性が無かった友達、鳴海紗良の代わりに、ノイ・ホワイトはバディアーマメント・オン...         4    401   
1  JESTA  魔法と科学が発展している世界の先進国のひとつであるアルバニオ大国では連続殺人事件が起こってい...         4    403   

                                             keyword  general_firstup  \
0    R15 残酷な描写あり VRMMO ほのぼの 女主人公 主人公最強 シリアス 配信 ご都合主義       1598344982   
1  R15 残酷な描写あり 異世界転生 異世界転移 IF戦記 ミステリー サスペンス サイコホラ...       1558531575   

   general_lastup  ...  review_cnt  all_point  all_hyoka_cnt  sasie_cnt  \
0      1598773036  ...           0        142             18          0   
1      1598773036  ...           0         40              5          0   

   kaiwaritu  novelupdated_at  updated_at  weekly_unique  \
0    

                                       title    ncode     userid     writer  \
0                                王～崩落する世界より～  N8243GJ  1974943.0       空色　一   
1  thou shalt kill &quot;WORLD JUSTICE&quot;  N9615EN  1260352.0  ビズ・リッキー・R   

                                               story  biggenre  genre  \
0  　ここは木の国。ここでは国民全員に手から木が出せる能力があった。その能力を授けてくれるのは木...         2    201   
1  「人の説く神が裁きを下さぬのなら、人の説く法が捌きを下さぬのなら、我等が神に代わり、正義に代...         2    201   

                                             keyword  general_firstup  \
0  R15 残酷な描写あり オリジナル戦記 冒険 異世界ファンタジー 恋愛 切ない 男主人公 西...       1595561177   
1  R15 残酷な描写あり 異世界転移 年の差 ハードボイルド サイコホラー 冒険 男主人公 ミ...       1517588373   

   general_lastup  ...  review_cnt  all_point  all_hyoka_cnt  sasie_cnt  \
0      1598773008  ...           0          0              0          0   
1      1598773008  ...           0         38              4          0   

   kaiwaritu  novelupdated_at  updated_at  weekly_unique  \
0         42       1598773008  15987

[2 rows x 41 columns]
                                              title    ncode     userid  \
0                                        オヤジVSメカオヤジ  N8889GL  1112122.0   
1  黒猫と契約したと思ったら黒衣の美少女になった件～ギルドで登録拒否されたけど強くなって見返します～  N6423GB   453320.0   

  writer                                              story  biggenre  genre  \
0  けにゃタン                                    オヤジー！\nでぇじょうぶか？        99   9999   
1     熊出  異世界に転移したら、そこは猫と契約を交わして魔物と戦う世界だった。\nギルドでお金を稼ごうと...         2    201   

                                             keyword  general_firstup  \
0                                                          1598773074   
1  残酷な描写あり 異世界転移 身分差 青春 異能力バトル ヒーロー 冒険 魔法 帝都十剣と七公...       1583543422   

   general_lastup  ...  review_cnt  all_point  all_hyoka_cnt  sasie_cnt  \
0      1598773074  ...           0          0              0          0   
1      1598773048  ...           0         38              5          0   

   kaiwaritu  novelupdated_at  updated_at  weekly_unique  \
0    

...

[2 rows x 41 columns]
  title ncode userid writer story biggenre genre keyword general_firstup  \
0  None         None   None  None     None  None    None            None   
1  None         None   None  None     None  None    None            None   

  general_lastup  ... review_cnt all_point all_hyoka_cnt sasie_cnt kaiwaritu  \
0           None  ...       None      None          None      None      None   
1           None  ...       None      None          None      None      None   

  novelupdated_at updated_at weekly_unique  text predict_point  
0            None       None          None  None          None  
1            None       None          None  None          None  

[2 rows x 41 columns]
  title ncode userid writer story biggenre genre keyword general_firstup  \
0  None         None   None  None     None  None    None            None   
1  None         None   None  None     None  None    None            None   

  general_lastup  ... review_cnt all_point all_hyoka_cnt sasi

[2 rows x 41 columns]
  title ncode userid writer story biggenre genre keyword general_firstup  \
0  None         None   None  None     None  None    None            None   
1  None         None   None  None     None  None    None            None   

  general_lastup  ... review_cnt all_point all_hyoka_cnt sasie_cnt kaiwaritu  \
0           None  ...       None      None          None      None      None   
1           None  ...       None      None          None      None      None   

  novelupdated_at updated_at weekly_unique  text predict_point  
0            None       None          None  None          None  
1            None       None          None  None          None  

[2 rows x 41 columns]
  title ncode userid writer story biggenre genre keyword general_firstup  \
0  None         None   None  None     None  None    None            None   
1  None         None   None  None     None  None    None            None   

  general_lastup  ... review_cnt all_point all_hyoka_cnt sasi

[2 rows x 41 columns]
  title ncode userid writer story biggenre genre keyword general_firstup  \
0  None         None   None  None     None  None    None            None   
1  None         None   None  None     None  None    None            None   

  general_lastup  ... review_cnt all_point all_hyoka_cnt sasie_cnt kaiwaritu  \
0           None  ...       None      None          None      None      None   
1           None  ...       None      None          None      None      None   

  novelupdated_at updated_at weekly_unique  text predict_point  
0            None       None          None  None          None  
1            None       None          None  None          None  

[2 rows x 41 columns]
  title ncode userid writer story biggenre genre keyword general_firstup  \
0  None         None   None  None     None  None    None            None   
1  None         None   None  None     None  None    None            None   

  general_lastup  ... review_cnt all_point all_hyoka_cnt sasi

[2 rows x 41 columns]
    title    ncode     userid  writer  \
0  アーモの門出  N7714GL   873785.0  KOLO×2   
1  HEBREW  N6885GK  1426763.0      右京   

                                               story  biggenre  genre  \
0  　味方からは『北の英雄』として讃えられ、敵からは『黒い悪魔』として恐れられてきたニジュ帝国の...         2    201   
1  時代は、戦火散る1800年代、舞台は、ホロコースト。激動の時代、動物ではなく同じ人間なのに大...         3    303   

                                             keyword  general_firstup  \
0  残酷な描写あり オリジナル戦記 伝奇 青春 異能力バトル ヒーロー ギャグ ダーク 男主人公...       1598796000   
1                                       時代小説 キネノベ大賞１       1598796000   

   general_lastup  ...  review_cnt  all_point  all_hyoka_cnt  sasie_cnt  \
0      1598796000  ...           0          0              0          0   
1      1598796000  ...           0          0              0          0   

   kaiwaritu  novelupdated_at  updated_at  weekly_unique  \
0         42       1598796003  1598796126              0   
1         38       1598796003  1598796126              0   

       

[2 rows x 41 columns]
         title    ncode     userid  writer  \
0  日本ワインに酔いしれて  N4103GI  1433087.0    三枝　優   
1     幼馴染に拾われて  N8392GF  1905976.0  夕時雨 のる   

                                               story  biggenre  genre  \
0  日本のワインがおいしくて、おいしくて。 それに合わせるおいしい料理も・・・ そんなお話です。...         1    102   
1  両親を亡くし家を追い出され途方に暮れる\n霜月(しもつき) 翔湊(かなた)は\n幼馴染で一人...         1    102   

                                             keyword  general_firstup  \
0             年の差 古典恋愛 ワイン 日本ワイン 酒 料理 グルメ 恋愛 旅行 酔っ払い       1593315694   
1  R15 スクールラブ 日常 青春 ラブコメ 幼馴染に拾われて ニヤニヤ 幼馴染 同棲 同居 ...       1589499407   

   general_lastup  ...  review_cnt  all_point  all_hyoka_cnt  sasie_cnt  \
0      1598795308  ...           0          8              1          0   
1      1598795267  ...           0         50              6          0   

   kaiwaritu  novelupdated_at  updated_at  weekly_unique  \
0         39       1598795308  1598795523            230   
1         72       1598795267  1598795406            

In [408]:
len(list(df_iterator))

0

In [409]:
df

,title,ncode,userid,writer,story,biggenre,genre,keyword,general_firstup,general_lastup,...,review_cnt,all_point,all_hyoka_cnt,sasie_cnt,kaiwaritu,novelupdated_at,updated_at,weekly_unique,text,predict_point
0,異界紀行～彷徨う亡霊と5つの奇蹟,N5535GL,1735273.0,うさ耳坊主,暗い過去を背負った青年、浅野春嗣。横断歩道で引かれそうになった女の子を助け死んでしまう。\n...,2,201,残酷な描写あり 異世界転生 異世界転移 シリアス ファンタジー,1598199911,1598795214,...,0,0,0,0,0,1598795867,1598796018,0,皆は考えた事が有るだろうか、この世界には違う世界・・・すなわち異世界がある事を。\n\n自分...,0
1,賢者の掟,N9139GL,2005274.0,絵空 ゴト,高校卒業したての平凡な顔と少しだけ聡明な頭脳を持ち合わせた、岩家 寿（イワイエ コトブキ）が...,3,304,残酷な描写あり 異世界転移 二次創作 ヒストリカル ミステリー 探偵小説,1598795212,1598795212,...,0,0,0,0,33,1598795212,1598795405,0,本、異世界、そして少女\n\n『知識』\n この世の全てがそれをもとに成り立つ。\n 文明...,0


In [410]:
conn.close()

In [416]:
conn, cursor = get_connector_and_cursor(DB_HOST_NAME)

In [431]:
df.head()

,title,ncode,userid,writer,story,biggenre,genre,keyword,general_firstup,general_lastup,...,review_cnt,all_point,all_hyoka_cnt,sasie_cnt,kaiwaritu,novelupdated_at,updated_at,weekly_unique,text,predict_point
0,異界紀行～彷徨う亡霊と5つの奇蹟,N5535GL,1735273.0,うさ耳坊主,暗い過去を背負った青年、浅野春嗣。横断歩道で引かれそうになった女の子を助け死んでしまう。\n...,2,201,残酷な描写あり 異世界転生 異世界転移 シリアス ファンタジー,1598199911,1598795214,...,0,0,0,0,0,1598795867,1598796018,0,皆は考えた事が有るだろうか、この世界には違う世界・・・すなわち異世界がある事を。\n\n自分...,0
1,賢者の掟,N9139GL,2005274.0,絵空 ゴト,高校卒業したての平凡な顔と少しだけ聡明な頭脳を持ち合わせた、岩家 寿（イワイエ コトブキ）が...,3,304,残酷な描写あり 異世界転移 二次創作 ヒストリカル ミステリー 探偵小説,1598795212,1598795212,...,0,0,0,0,33,1598795212,1598795405,0,本、異世界、そして少女\n\n『知識』\n この世の全てがそれをもとに成り立つ。\n 文明...,0


In [434]:
a,b = df[['text', 'story']]

In [437]:
df[['text', 'story']]

,text,story
0,皆は考えた事が有るだろうか、この世界には違う世界・・・すなわち異世界がある事を。\n\n自分...,暗い過去を背負った青年、浅野春嗣。横断歩道で引かれそうになった女の子を助け死んでしまう。\n...
1,本、異世界、そして少女\n\n『知識』\n この世の全てがそれをもとに成り立つ。\n 文明...,高校卒業したての平凡な顔と少しだけ聡明な頭脳を持ち合わせた、岩家 寿（イワイエ コトブキ）が...


In [451]:
client = Elasticsearch(ELASTICSEARCH_HOST_NAME)
res = client.search(
    index='features',
    body={
        'query': {'match_all': {}},
    }
)

In [452]:
res['hits']['hits'][0]

{'_index': 'features',
 '_type': '_doc',
 '_id': 'B0rwQXQBTuACM86QV9jO',
 '_score': 1.0,
 '_source': {'ncode': 'N3934GL',
  'writer': '竹井茂',
  'keyword': 'R15 残酷な描写あり 異世界転生 冒険 男主人公 ロボット',
  'story': '目が覚めたら土の中でした。死ぬかと思ったけど、大丈夫、生きてます。\nえ？この世界はロボも銃も魔法もあんの？\n\nよっしゃぁ！死ぬほど楽しんでやるぜぇ！\n\nエイリアンに侵略された異世界を生きるお話。',
  'feature': [-1.0238600969314575,
   0.9841390252113342,
   1.588611125946045,
   0.759050726890564,
   -0.9123788475990295,
   -0.11882445216178894,
   1.7227530479431152,
   -0.6846081018447876,
   0.9473816156387329,
   -1.3995535373687744,
   2.2016372680664062,
   -2.1541600227355957,
   -0.22616833448410034,
   -1.1347771883010864,
   1.5486160516738892,
   0.9584373831748962,
   -0.2363678365945816,
   0.7105288505554199,
   -1.253975749015808,
   1.3455382585525513,
   1.638058066368103,
   -1.0350350141525269,
   -3.060502290725708,
   -0.9751853942871094,
   0.23718321323394775,
   -0.4084985852241516,
   -0.4048694968223572,
   1.9966821670532227,
   1.58939528465271,
   0.4